In [ ]:
import json
import os
from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import unary_union

In [ ]:
DATA_DIR = './drive/MyDrive/ajour/'
modify_dir=os.path.join(DATA_DIR, 'fillSingleJersey')
modify_list=os.listdir(modify_dir)
label_dir=os.path.join(DATA_DIR, 'label')
another_dir=os.path.join(DATA_DIR, 'noFill')
another_dir2=os.path.join(DATA_DIR, 'fillPurl')
another_list=os.listdir(another_dir)
another_list2=os.listdir(another_dir2)

In [ ]:
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt
import os

# 색상 정의 (category_id별 색상)
category_colors = {
    4: (0, 255, 255),  # 청록 -> single_jersey
    1: (255, 0, 0),  # 빨강->rib
    2: (0, 255, 0),  # 초록-> pow(purl)
    3: (0, 0, 255),  # 파랑
    0: (255, 255, 0), # 노랑->type3
    5: (255, 20, 147), #핑크,
    6: (0,0,0) #black
}

def fill_unlabeled_areas_with(json_data, canvas_size):
    # 캔버스 생성 (검은색 배경: 0)
    image = np.zeros((canvas_size[1], canvas_size[0],3), dtype=np.uint8)

    #만약 빈 부분 다른 라벨로 채운다면 여기에 RGB to BGR 변환해서 넣기기
    # for i in range(canvas_size[1]):
    #     for j in range(canvas_size[0]):
    #         # 순서 거꾸로 넣기
    #         image[i][j][0]=255
    #         image[i][j][1]=255
    #         image[i][j][2]=0

    label_name=json_data["imagePath"]

    #1차로 nothing부터 검은색으로 채우기- Fill with Single Jersey
    # for annotation in json_data["shapes"]:
    #     # category_id = annotation["category_id"]
    #     if(annotation["label"]=="nothing"):
    #       category_id=6
    #     else:
    #       continue
    #     polygon = annotation["points"]

    #     # 색상 선택
    #     color = category_colors.get(category_id, (255, 255, 255))  # 기본 색상: 흰색

    #     # 다각형 좌표를 픽셀 단위로 변환
    #     points = np.array([
    #         [int(x ), int(y)] for x, y in polygon
    #     ], dtype=np.int32)

    #     # 다각형 그리기
    #     cv2.polylines(image, [points], isClosed=True, color=color, thickness=2)
    #     cv2.fillPoly(image, [points], color=color)

    # 다각형 그리기 - 추가로 이용한 라벨 있으면 수정해서 넣기
    for annotation in json_data["shapes"]:
        # category_id = annotation["category_id"]
        if(annotation["label"]=="single_jersey"):
          category_id=0
        if(annotation["label"]=="single jersey"):
          category_id=0
        if(annotation["label"]=="rib"):
          category_id=1
        if(annotation["label"]=="purl"):
          category_id=2
        if(annotation["label"]=="ajour"):
          category_id=4
        if(annotation["label"]=="nothing"):
          continue
        polygon = annotation["points"]

        # 색상 선택
        color = category_colors.get(category_id, (255, 255, 255))  # 기본 색상: single_jersey 색상
        # 다각형 좌표를 픽셀 단위로 변환
        points = np.array([
            [int(x ), int(y)] for x, y in polygon
        ], dtype=np.int32)

        # 다각형 그리기
        cv2.polylines(image, [points], isClosed=True, color=color, thickness=2)
        cv2.fillPoly(image, [points], color=color)

    # 이미지를 Matplotlib으로 출력
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    output_path = os.path.join(label_dir, f"annotated_{label_name}")

    # 이미지 저장
    cv2.imwrite(output_path, image)

In [ ]:
for i in modify_list:
    input_file_path = os.path.join(modify_dir,i)
    with open(input_file_path, 'r') as file:
        json_data = json.load(file)
        canvas_size=(json_data["imageWidth"],json_data["imageHeight"])
        updated_json = fill_unlabeled_areas_with(json_data, canvas_size)
        # print(updated_json)

        # output_file_path = os.path.join(modify_dir,"updated_{}".format(i))
        # with open(output_file_path, 'w') as file:
        #     json.dump(updated_json, file, indent=2)

In [ ]:
label_list=os.listdir(label_dir)
print(len(label_list))

In [ ]:
image_dir = os.path.join(DATA_DIR, 'image')
image_list=os.listdir(image_dir)
print(len(image_list))

여기서부터는 이미지가 있는데 라벨이 없는 것을 확인하기 위한 코드

In [ ]:
a=[]
for i in range(220):
  if "{}.jpg".format(i) in image_list:
    # if("annotated_{}.jpg".format(i) in label_list):
    #   continue
    # else:
    a.append(i)
print(a)

In [ ]:
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt
import os

image_dir = "./image"
json_dir = "./json"
# 색상 정의 (category_id별 색상)
category_colors = {
    4: (0, 255, 255),  # 청록 -> single_jersey
    1: (255, 0, 0),  # 빨강->rib
    2: (0, 255, 0),  # 초록-> pow(purl)
    3: (0, 0, 255),  # 파랑
    0: (255, 255, 0), # 노랑->type3
    5: (255, 20, 147), #핑크,
    6: (0,0,0) #black
}
image_list=os.listdir(image_dir)
json_list=os.listdir(json_dir)

for i in json_list:
    # print(i)
    input_file_path = os.path.join(json_dir,i)
    with open(input_file_path, 'r') as file:
        json_data = json.load(file)
    # 이미지 파일 이름 추출
    if(len(json_data["shapes"])==0):
        continue
    # print(json_data["annotations"][0]["image_name"]

    image_name = json_data["imagePath"]
    # 이미지 로드
    image = cv2.imread(os.path.join("image",image_name))
    if image is None:
        print(f"이미지를 찾을 수 없습니다:")
        exit()
    # print(image)
    # 이미지 크기
    height, width, _ = image.shape

    #1차로 nothing부터 검은색으로 채우기
    for annotation in json_data["shapes"]:
        # category_id = annotation["category_id"]
        if(annotation["label"]=="nothing"):
          category_id=6
        else:
          continue
        polygon = annotation["points"]

        # 색상 선택
        color = category_colors.get(category_id, (255, 255, 255))  # 기본 색상: 흰색

        # 다각형 좌표를 픽셀 단위로 변환
        points = np.array([
            [int(x ), int(y)] for x, y in polygon
        ], dtype=np.int32)

        # 다각형 그리기
        cv2.polylines(image, [points], isClosed=True, color=color, thickness=2)
        cv2.fillPoly(image, [points], color=color)

    # 다각형 그리기
    for annotation in json_data["shapes"]:
        # category_id = annotation["category_id"]
        if(annotation["label"]=="single_jersey"):
          category_id=0
        if(annotation["label"]=="rib"):
          category_id=1
        if(annotation["label"]=="pow"):
          category_id=2
        if(annotation["label"]=="type3"):
          category_id=4
        if(annotation["label"]=="nothing"):
          continue
        polygon = annotation["points"]

        # 색상 선택
        color = category_colors.get(category_id, (255, 255, 255))  # 기본 색상: 흰색

        # 다각형 좌표를 픽셀 단위로 변환
        points = np.array([
            [int(x ), int(y)] for x, y in polygon
        ], dtype=np.int32)

        # 다각형 그리기
        cv2.polylines(image, [points], isClosed=True, color=color, thickness=2)
        cv2.fillPoly(image, [points], color=color)

    # 이미지를 Matplotlib으로 출력
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    output_path = os.path.join("label", f"annotated_{image_name}.jpg")

    # 이미지 저장
    cv2.imwrite(f"label/annotated_{image_name}", image)

In [ ]:
lists=[1, 2, 3, 5, 6, 8, 9, 10, 13, 14, 16, 19, 20, 21, 22, 25, 27, 28, 31, 32, 35, 36, 37, 38, 39, 40, 41, 43, 45, 46, 47, 49, 50, 51, 52, 53, 57, 59, 60, 64, 65, 66, 67, 69, 70, 71, 72, 73, 75, 76, 77, 78, 79, 81, 82, 83, 84, 86, 87, 89, 92, 93, 94, 95, 96, 98, 102, 103, 105, 106, 107, 108, 109, 110, 111, 112, 114, 115, 116, 120, 122, 123, 127, 128, 129, 130, 131, 132, 133, 135, 136, 137, 138, 139, 140, 141, 142, 143, 145, 146, 148, 150, 156, 159, 160, 161, 162, 164, 166, 167, 168, 169, 170, 171, 172, 173, 174, 176, 177, 178, 179, 181, 182, 183, 184, 185, 187, 188, 190, 193, 194, 196, 197, 198]
down=[64,75,156,174]
center=[1,35,59,98,106,129,135,172,179,197]

정사각형으로 자르고 256,256 resizing

In [ ]:
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt
import os
for i in lists:
    # input_file_path = 'image_{}.json'.format(i)
    # with open(input_file_path, 'r') as file:
    #     json_data = json.load(file)
    # # 이미지 파일 이름 추출
    # if(len(json_data["annotations"])==0):
    #     continue
    # # print(json_data["annotations"][0]["image_name"]

    image_name = "{}.jpg".format(i)
    # 이미지 로드
    image = cv2.imread(os.path.join(DATA_DIR,"image",image_name))

    if image is None:
        print(f"이미지를 찾을 수 없습니다: {image_name}")
        exit()

    # 이미지 크기
    height, width, _ = image.shape
    # image_name = "{}.jpg".format(i)
    output_path1 = os.path.join(DATA_DIR,"cropped/image",image_name)
    image2 = cv2.imread((os.path.join(DATA_DIR,"label","annotated_{}.jpg".format(i))))
    output_path2 = os.path.join(DATA_DIR,"cropped/label","annotated_{}.jpg".format(i))

    # 정사각형 크기 결정 (가로와 세로 중 작은 값 선택)
    square_size = min(height, width)

    # 중앙 기준으로 정사각형 크롭 좌표 계산
    start_x = (width - square_size) // 2
    end_x = start_x + square_size
    if i in down:
      start_y = height-square_size
      end_y = height
    elif i in center:
      start_y = (height - square_size) // 2
      end_y = square_size + start_y
    else:
      start_y = 0
      end_y = square_size
    print(i,": ",square_size)
    # 이미지 크롭
    cropped_img1 = image[start_y:end_y, start_x:end_x]
    cropped_img2 = image2[start_y:end_y, start_x:end_x]
    cropped_img1=cv2.resize(cropped_img1,(256,256))
    cropped_img2=cv2.resize(cropped_img2,(256,256))


    # 이미지 저장
    cv2.imwrite(output_path1, cropped_img1)
    cv2.imwrite(output_path2, cropped_img2)